# Challenge 2

In this challenge we will continue working with the `Pokemon` dataset. We will attempt solving a slightly more complex problem in which we will practice the iterative data analysis process you learned in [this video](https://www.youtube.com/watch?v=xOomNicqbkk).

The problem statement is as follows:

**You are at a Pokemon black market planning to buy a Pokemon for battle. All Pokemon are sold at the same price and you can only afford to buy one. You cannot choose which specific Pokemon to buy. However, you can specify the type of the Pokemon - one type that exists in either `Type 1` or `Type 2`. Which type should you choose in order to maximize your chance of receiving a good Pokemon?**

To remind you about the 3 steps of iterative data analysis, they are:

1. Setting Expectations
1. Collecting Information
1. Reacting to Data / Revising Expectations

Following the iterative process, we'll guide you in completing the challenge.


## Problem Solving Iteration 1

In this iteration we'll analyze the problem and identify the breakthrough. The original question statement is kind of vague because we don't know what a *good pokemon* really means as represented in the data. We'll start by understanding the dataset and see if we can find some insights.

In [20]:
# Import libraries
import numpy as np
import pandas as pd

In [21]:
# Importing the dataset
pokemon = pd.read_csv("Pokemon.csv")
pokemon.columns


Index(['#', 'Name', 'Type 1', 'Type 2', 'Total', 'HP', 'Attack', 'Defense',
       'Sp. Atk', 'Sp. Def', 'Speed', 'Generation', 'Legendary'],
      dtype='object')

From the data it seems whether a pokemon is good depends on its abilities as represented in the fields of `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total`. We are not sure about `Generation` and `Legendary` because they are not necessarily the decisive factors of the pokemon abilities.

But `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total` are a lot of fields! If we look at them all at once it's very complicated. This isn't Mission Impossible but it's ideal that we tackle this kind of problem after we learn Machine Learning (which you will do in Module 3). For now, is there a way to consolidate the fields we need to look into?

Fortunately there seems to be a way. It appears the `Total` field is computed based on the other 6 fields. But we need to prove our theory. If we can approve there is a formula to compute `Total` based on the other 6 abilities, we only need to look into `Total`.

We have the following expectation now:

#### The `Total` field is computed based on `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, and `Speed`.

We need to collect the following information:

* **What is the formula to compute `Total`?**
* **Does the formula work for all pokemon?**

In the cell below, make a hypothesis on how `Total` is computed and test your hypothesis.

In [22]:
# your code here you dont want to drop any column
pokemon[['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed',"Total"]]


,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Total
0,45,49,49,65,65,45,318
1,60,62,63,80,80,60,405
2,80,82,83,100,100,80,525
3,80,100,123,122,120,80,625
4,39,52,43,60,50,65,309
...,...,...,...,...,...,...,...
795,50,100,150,100,150,50,600
796,50,160,110,160,110,110,700
797,80,110,60,150,130,70,600
798,80,160,60,170,130,80,680


In [26]:
# well i got kinda dissapointed,it just sums everything ,but we also want to know if it works for all pokemon
pokemon["Computed_Total"] = pokemon[['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']].sum(axis=1)

#and now check if every match is a true
all_totals_match = (pokemon["Computed_Total"] ==pokemon["Total"]).all()
all_totals_match

True

In [27]:
#ok so they are not all the same ,so i should filter to see which one is the IMPOSTOR
not_same_total = pokemon[pokemon["Computed_Total"] != pokemon["Total"]]
not_same_total[['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed',"Total"]]
# fun fact i included Total in the first calculation and thats why i was given more in total than in computed_total,but after more researching i find it out .

,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Total


In [29]:
#Does the formula work for all pokemon? Yes

## Problem Solving Iteration 2

Now that we have consolidated the abilities fields, we can update the problem statement. The new problem statement is:

### Which pokemon type is most likely to have the highest `Total` value?

In the updated problem statement, we assume there is a certain relationship between the `Total` and the pokemon type. But we have two *type* fields (`Type 1` and `Type 2`) that have string values. In data analysis, string fields have to be transformed to numerical format in order to be analyzed. 

In addition, keep in mind that `Type 1` always has a value but `Type 2` is sometimes empty (having the `NaN` value). Also, the pokemon type we choose may be either in `Type 1` or `Type 2`.

Now our expectation is:

#### `Type 1` and `Type 2` string variables need to be converted to numerical variables in order to identify the relationship between `Total` and the pokemon type.

The information we need to collect is:

#### How to convert two string variables to numerical?

Let's address the first question first. You can use a method called **One Hot Encoding** which is frequently used in machine learning to encode categorical string variables to numerical. The idea is to gather all the possible string values in a categorical field and create a numerical field for each unique string value. Each of those numerical fields uses `1` and `0` to indicate whether the data record has the corresponding categorical value. A detailed explanation of One Hot Encoding can be found in [this article](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f). You will formally learn it in Module 3.

For instance, if a pokemon has `Type 1` as `Poison` and `Type 2` as `Fire`, then its `Poison` and `Fire` fields are `1` whereas all other fields are `0`. If a pokemon has `Type 1` as `Water` and `Type 2` as `NaN`, then its `Water` field is `1` whereas all other fields are `0`.

#### In the next cell, use One Hot Encoding to encode `Type 1` and `Type 2`. Use the pokemon type values as the names of the numerical fields you create.

The new numerical variables you create should look like below:

![One Hot Encoding](../images/one-hot-encoding.png)

In [43]:
# your code here
type1_encoded = pd.get_dummies(pokemon["Type 1"], prefix = "", prefix_sep ="") # by using prefix empty string we asure that we get only the name of the types 
type2_encoded = pd.get_dummies(pokemon["Type 2"], prefix = "", prefix_sep ="") 

one_hot_encoded = type1_encoded | type2_encoded # so it seems that if you combine both dataframes with a OR statement if you get 2 True values ,2 ones you add them both this way,for example
# Example 
# type1_encoded has values: {'Fire': 1, 'Water': 0, 'Grass': 0}
# type2_encoded has values: {'Fire': 0, 'Water': 1, 'Grass': 0}
# this would result in {'Fire': 1, 'Water': 1, 'Grass': 0}

# now append the one hot encoded columns to the original pokemon df
pokemon_encoded = pd.concat([pokemon,one_hot_encoded],axis = 1)
pokemon_encoded
one_hot_encoded

,Bug,Dark,Dragon,Electric,Fairy,Fighting,Fire,Flying,Ghost,Grass,Ground,Ice,Normal,Poison,Psychic,Rock,Steel,Water
0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
796,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
797,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0
798,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


## Problem Solving Iteration 3

Now we have encoded the pokemon types, we will identify the relationship between `Total` and the encoded fields. Our expectation is:

#### There are relationships between `Total` and the encoded pokemon type variables and we need to identify the correlations.

The information we need to collect is:

#### How to identify the relationship between `Total` and the encoded pokemon type fields?

There are multiple ways to answer this question. The easiest way is to use correlation. In the cell below, calculate the correlation of `Total` to each of the encoded fields. Rank the correlations and identify the #1 pokemon type that is most likely to have the highest `Total`.

In [33]:
# your code here
correlations = one_hot_encoded.corrwith(pokemon["Total"]) # if you use "corrwith" you see only the correlation of that data with total(another data)
correlations

Bug        -0.145781
Dark        0.056154
Dragon      0.229705
Electric    0.020971
Fairy      -0.036698
Fighting    0.077786
Fire        0.078726
Flying      0.059383
Ghost       0.003641
Grass      -0.052592
Ground      0.015060
Ice         0.060248
Normal     -0.105331
Poison     -0.090441
Psychic     0.124688
Rock        0.032731
Steel       0.109703
Water      -0.021665
dtype: float64

In [39]:
# thats not too clear ,lets order this
correlations_sorted = correlations.sort_values(ascending = False)
correlations_sorted


Dragon      0.229705
Psychic     0.124688
Steel       0.109703
Fire        0.078726
Fighting    0.077786
Ice         0.060248
Flying      0.059383
Dark        0.056154
Rock        0.032731
Electric    0.020971
Ground      0.015060
Ghost       0.003641
Water      -0.021665
Fairy      -0.036698
Grass      -0.052592
Poison     -0.090441
Normal     -0.105331
Bug        -0.145781
dtype: float64

## Conclusion
And there you go ,seems like a Dragon type is x2 more correlated to our target than the top 2 which is psychic,so if we wanted to have better chances to get a good pokemon based only on his type we just pick a Dragon which lets be honest is pretty obvious ,you wont see khalesy(Daenerys Targaryen) from games of thrones fly ove a Bug which is last on the list,a dragon really sounds like the obvious choice.


# Bonus Question

Say now you can choose both `Type 1` and `Type 2` of the pokemon. In order to receive the best pokemon, which types will you choose?

In [42]:
# your code here
combined_types = pokemon.groupby(["Type 1" , "Type 2"]).mean()["Total"].sort_values(ascending=False)
combined_types

C:\Users\Admin\AppData\Local\Temp\ipykernel_29240\1826111198.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  combined_types = pokemon.groupby(["Type 1" , "Type 2"]).mean()["Total"].sort_values(ascending=False)


Type 1   Type 2
Ground   Fire      770.000000
Dragon   Ice       686.666667
Steel    Dragon    680.000000
Ghost    Dragon    680.000000
Psychic  Dark      680.000000
                      ...    
Poison   Bug       330.000000
Normal   Fairy     330.000000
Poison   Water     320.000000
Bug      Water     269.000000
         Ghost     236.000000
Name: Total, Length: 136, dtype: float64

## Conclusion 
So surprise surprise over here,looks like if u can chose  a double type pokemon you get as a winner Ground fire types to be your best choice.
### explanation of code 
* the groupby ,groups the dataframe pokemon by type 1 and type 2 .This means that for every unique pair of types like fire and flying or Water and Nan a separate group is created.(this would also get us know that double types are usally better than mono types),btw it only gets unique combinations so no dragon/dragon types.
* the mean is to calculate the average of all numeric columns for each group,so for each unique combination of type 1 and type 2 we get the average values of all numeric columns including total, hp,attack,etc.
* then we only get Total since we only are interested on that one  and we sort the results.